In [63]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import joblib  # Import joblib to save and load LabelEncoder

In [64]:
# Directory paths
base_dir = r"C:\Sign_Language_Detection\split_dataset"
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

# Function to load data from .npy files in a directory
def load_data_from_directory(directory):
    gestures = os.listdir(directory)
    data = []
    labels = []
    for gesture in gestures:
        gesture_dir = os.path.join(directory, gesture)
        if os.path.isdir(gesture_dir):
            files = os.listdir(gesture_dir)
            for file in files:
                if file.endswith('.npy'):
                    file_path = os.path.join(gesture_dir, file)
                    image_data = np.load(file_path, allow_pickle=True)
                    data.append(image_data)  # Assuming your .npy file contains image data
                    labels.append(gesture)   # Label is the name of the gesture folder
    return np.array(data), np.array(labels)

# Load training and validation data
train_data, train_labels = load_data_from_directory(train_dir)
val_data, val_labels = load_data_from_directory(val_dir)

# Example of data structure
print(f"Train data shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Validation data shape: {val_data.shape}, Validation labels shape: {val_labels.shape}")


Train data shape: (253, 21, 3), Train labels shape: (253,)
Validation data shape: (74, 21, 3), Validation labels shape: (74,)


In [65]:
train_labels

array(['call', 'call', 'call', 'call', 'call', 'call', 'call', 'call',
       'call', 'call', 'call', 'call', 'call', 'call', 'call', 'call',
       'call', 'call', 'call', 'call', 'call', 'call', 'call', 'call',
       'dislike', 'dislike', 'dislike', 'dislike', 'dislike', 'dislike',
       'dislike', 'dislike', 'dislike', 'dislike', 'dislike', 'dislike',
       'dislike', 'dislike', 'dislike', 'dislike', 'dislike', 'dislike',
       'dislike', 'dislike', 'dislike', 'dislike', 'dislike', 'dislike',
       'fist', 'fist', 'fist', 'fist', 'fist', 'fist', 'fist', 'fist',
       'fist', 'fist', 'fist', 'fist', 'fist', 'fist', 'fist', 'fist',
       'fist', 'fist', 'fist', 'fist', 'fist', 'fist', 'fist', 'fist',
       'four', 'four', 'four', 'four', 'four', 'four', 'four', 'four',
       'four', 'four', 'four', 'four', 'four', 'four', 'four', 'four',
       'four', 'four', 'four', 'four', 'four', 'four', 'four', 'four',
       'like', 'like', 'like', 'like', 'like', 'like', 'like', 'like'

In [66]:
encoder = LabelEncoder()
encoder.fit(train_labels)
train_encoded_labels = encoder.transform(train_labels)
val_encoded_labels = encoder.transform(val_labels)

# One-hot encoding
onehot_encoder = OneHotEncoder(sparse_output=False)
train_onehot_labels = onehot_encoder.fit_transform(train_encoded_labels.reshape(-1, 1))
val_onehot_labels = onehot_encoder.transform(val_encoded_labels.reshape(-1, 1))

# Shapes after one-hot encoding
print(f"Train labels shape after encoding: {train_onehot_labels.shape}")
print(f"Validation labels shape after encoding: {val_onehot_labels.shape}")


Train labels shape after encoding: (253, 11)
Validation labels shape after encoding: (74, 11)


In [67]:
train_onehot_labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [68]:
val_onehot_labels

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

In [69]:
train_data = train_data.astype('float32') / 255.0
val_data = val_data.astype('float32') / 255.0

In [70]:
train_data = train_data.reshape(-1, 21, 3, 1)
val_data = val_data.reshape(-1, 21, 3, 1)

In [71]:
# Example of data structure
print(f"Train data shape: {train_data.shape}, Validation data shape: {val_data.shape}")

Train data shape: (253, 21, 3, 1), Validation data shape: (74, 21, 3, 1)


In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Create a Sequential model
model = Sequential()

# Flatten the input (assuming your data shape is (batch_size, 21, 3))
model.add(Flatten(input_shape=(21, 3)))  # Adjust input_shape if needed

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))

# Output layer
model.add(Dense(11, activation='softmax'))  # Adjust output units according to your classification needs

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary to check layer dimensions
model.summary()

c:\Sign_Language_Detection\env\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_5 (Flatten)             │ (None, 63)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 512)            │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 198,411 (775.04 KB)

 Trainable params: 198,411 (775.04 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
from keras.callbacks import EarlyStopping

# Assuming model is already defined and compiled
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

history = model.fit(train_data, train_onehot_labels, 
                    epochs=500, 
                    validation_data=(val_data, val_onehot_labels),
                    callbacks=[early_stopping])


Epoch 1/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.1072 - loss: 2.3977 - val_accuracy: 0.0946 - val_loss: 2.3967
Epoch 2/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0911 - loss: 2.3953 - val_accuracy: 0.0946 - val_loss: 2.3953
Epoch 3/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0847 - loss: 2.3953 - val_accuracy: 0.0946 - val_loss: 2.3941
Epoch 4/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1116 - loss: 2.3898 - val_accuracy: 0.0946 - val_loss: 2.3929
Epoch 5/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0985 - loss: 2.3905 - val_accuracy: 0.0946 - val_loss: 2.3922
Epoch 6/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0872 - loss: 2.3976 - val_accuracy: 0.0946 - val_loss: 2.3916
Epoch 7/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0757 - loss: 2.3952 - val_accuracy: 0.0946 - val_loss: 2.3910
Epoch 8/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0937 - loss: 2.3935 - val_accuracy: 0.0946 - val_loss:

In [ ]:
# Save the fitted encoder
joblib.dump(encoder, 'label_encoder.pkl')

In [75]:
joblib.dump(onehot_encoder, 'onehot_encoder.pkl')


['onehot_encoder.pkl']

In [76]:
model.save('sign_language_model.h5')
